In [1]:
import xarray as xr
import pandas as pd
import netCDF4 as nc
import tqdm
import csv



In [2]:
filePath = '../Dataset/climate_dataset/TempAir.nc'  
dSet = nc.Dataset(filePath)  

In [3]:
for dimension in dSet.dimensions.values():  
    print(dimension)  

<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 744
<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 720
<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 360


In [6]:
import netCDF4 as nc
import csv
import tqdm
import xarray as xr

ds = xr.open_dataset("../Dataset/climate_dataset/TempAir.nc")
nc_file = ds

csv_file = open('output.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

time = nc_file.variables['time']
lon = nc_file.variables['lon']
lat = nc_file.variables['lat']
csv_writer.writerow(['Time', 'Lon', 'Lat', 'Value'])

data_variable = nc_file.variables['Tair']

# Reduce the size of the 'time' dimension to 100
time_indices = range(100, 130)

for t in tqdm.tqdm(time_indices):
    for lat_index in range(len(lat)):
        for lon_index in range(len(lon)):
            time_value = time[t]
            lat_value = lat[lat_index]
            lon_value = lon[lon_index]
            data_value = data_variable[t, lat_index, lon_index]
            csv_writer.writerow([time_value.values, lon_value.values, lat_value.values, data_value.values])


csv_file.close()
nc_file.close()



100%|██████████| 30/30 [2:56:48<00:00, 353.61s/it]   


In [7]:
df=pd.read_csv('output.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7776000 entries, 0 to 7775999
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Time    object 
 1   Lon     float64
 2   Lat     float64
 3   Value   float64
dtypes: float64(3), object(1)
memory usage: 237.3+ MB


In [9]:
train=df.drop(columns=['Time'])

In [10]:
train

,Lon,Lat,Value
0,-179.75,-89.75,245.06471
1,-179.25,-89.75,245.06508
2,-178.75,-89.75,245.07979
3,-178.25,-89.75,245.08086
4,-177.75,-89.75,245.09454
...,...,...,...
7775995,177.75,89.75,NaN
7775996,178.25,89.75,NaN
7775997,178.75,89.75,NaN
7775998,179.25,89.75,NaN


In [11]:
df['Time']=pd.to_datetime(df['Time'])


In [100]:
#!pip install sklearn==1.2.1

In [12]:

import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

In [13]:
from missingpy import MissForest
imputer = MissForest(n_estimators=100, max_iter=10, n_jobs=-1)
X_imputed = imputer.fit_transform(train)

/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Iteration: 0


/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 1


/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 2


/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 3


/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/Users/farahmurtaza/Library/Python/3.9/lib/python/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 4


In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(X_imputed)

In [16]:
df_final_data=pd.DataFrame(scaled_data,columns=['Lon','Lat','Value'])

In [17]:
df_final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7776000 entries, 0 to 7775999
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Lon     float64
 1   Lat     float64
 2   Value   float64
dtypes: float64(3)
memory usage: 178.0 MB


In [18]:
df_final_data['Time']=df['Time']

In [19]:
df_final_data

,Lon,Lat,Value,Time
0,-1.729647,-1.727246,-1.556229,2019-01-05 04:00:00
1,-1.724836,-1.727246,-1.556211,2019-01-05 04:00:00
2,-1.720024,-1.727246,-1.555488,2019-01-05 04:00:00
3,-1.715213,-1.727246,-1.555436,2019-01-05 04:00:00
4,-1.710402,-1.727246,-1.554764,2019-01-05 04:00:00
...,...,...,...,...
7775995,1.710402,1.727246,-1.462160,2019-01-06 09:00:00
7775996,1.715213,1.727246,-1.462160,2019-01-06 09:00:00
7775997,1.720024,1.727246,-1.462160,2019-01-06 09:00:00
7775998,1.724836,1.727246,-1.427818,2019-01-06 09:00:00


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
# Split the data based on the 'time' column
train_data, test_data = train_test_split(df_final_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.3125, random_state=42)

train_file_path = "../Dataset/climate_dataset/train_data.csv"
validation_file_path = "../Dataset/climate_dataset/validation_data.csv"
test_file_path = "../Dataset/climate_dataset/test_data.csv"


# Save the datasets to CSV files
train_data.to_csv(train_file_path, index=False)
val_data.to_csv(validation_file_path, index=False)
test_data.to_csv(test_file_path, index=False)
